In [65]:
import os, sys, inspect
import time

lib_folder = os.path.join(os.path.split(inspect.getfile( inspect.currentframe() ))[0], 'CapableRobot_USBHub_Driver')
lib_load = os.path.realpath(os.path.abspath(lib_folder))

if lib_load not in sys.path:
    sys.path.insert(0, lib_load)

import capablerobot_usbhub 

SIM = False

# hub = capablerobot_usbhub.USBHub()
# hub.i2c.enable()

try:
    hub = capablerobot_usbhub.USBHub()
    hub.i2c.enable()
except ValueError:
    SIM = True

In [66]:
import time

import paho.mqtt.client as mqtt
import bqplot as bq
import numpy as np
import pandas as pd

from IPython.display import display

def live_plot(title, fields, x="", y="", ylimit=None, history=50):
    global drop_count
    drop_count=0
    
    x_sc = bq.LinearScale()
    y_sc = bq.LinearScale()
    
    if ylimit is not None:
        y_sc.min = ylimit[0]
        y_sc.max = ylimit[1]

    ax_x = bq.Axis(label=x, scale=x_sc, grid_lines='solid')
    ax_y = bq.Axis(label=y, scale=y_sc, orientation='vertical', grid_lines='solid')

    df = pd.DataFrame(columns=['time']+fields)
        
    pts = bq.Lines(x=df.index, y=[df[f].values for f in fields], display_legend=True, scales={'x': x_sc, 'y': y_sc})
    fig = bq.Figure(axes=[ax_x, ax_y], marks=[pts], labels=fields, legend_location='top-left', title=title)
    
    def cb(data):
        global drop_count
        
        if len(df) > history:
            df.drop([drop_count], inplace=True)
            drop_count += 1
        
        df.loc[len(df)+drop_count] = [time.time()] + data
        
        with pts.hold_sync():
            pts.x = df.index 
            pts.y = [df[f] for f in fields]
    
    return fig, cb

In [67]:
def print_currents():
    last = int(time.time() * 1000)
    
    while True:
        now = int(time.time() * 1000)
        print(str(now - last).rjust(3), " ".join([("%.2f" % v).rjust(7) for v in hub.power.measurements()]))
        last = now
        time.sleep(0.5)

In [68]:
import ipywidgets as widgets
import copy
import random

out = widgets.Output()
display(out)

@out.capture()
def power_change(change):
    port = int(change['owner'].description.split(" ")[1])
    
    if SIM:
        if change['new']:
            print("POWER ENABLE {}".format(port))
        else:
            print("POWER DISABLE {}".format(port))
    else:
        if change['new']:
            hub.power.enable(ports=[port])
        else:  
            hub.power.disable(ports=[port])

@out.capture()
def data_change(change):
    port = int(change['owner'].description.split(" ")[1])
    
    if SIM:
        if change['new']:
            print("DATA ENABLE {}".format(port))
        else:
            print("DATA DISABLE {}".format(port))
    else:
        if change['new']:      
            hub.data_enable(ports=[port])
        else:  
            hub.data_disable(ports=[port])

def setup_ui():
    power_buttons = []
    data_buttons = []
    
    for idx in [1,2,3,4]:
        label = "Port {}".format(idx)
        
        button = widgets.ToggleButton(description=label, disabled=False, value=True)
        button.observe(power_change, names='value')
        power_buttons.append(button)
        
        button = widgets.ToggleButton(description=label, disabled=False, value=True)
        button.observe(data_change, names='value')
        data_buttons.append(button)
    
    power_label = widgets.Label(value="Power", layout=widgets.Layout(width='20%'))
    power_row   = widgets.HBox([power_label]+power_buttons)
    
    data_label  = widgets.Label(value="Data", layout=widgets.Layout(width='20%'))
    data_row    = widgets.HBox([data_label]+data_buttons)
    
    display(widgets.VBox([data_row, power_row]))        
    
def graph_currents(do_stop):
    fields = ["1", "2", "3", "4"]
    lim = 1700
    if SIM:
        lim = 10
    fig, cb = live_plot("Port Current", fields, x="Time (sec)", y="Current (mA)", ylimit=[0,lim])

    display(fig)
    
    while True:
        try:
            if SIM:
                cb([random.random(),random.random(),random.random(),random.random()])
            else:
                cb(hub.power.measurements())
        except USBError:
            pass

        time.sleep(0.2)
        
        if do_stop():
            break

Output()

In [69]:
# print_currents()

setup_ui()

stop_threads = False

def stop():
    global stop_threads
    stop_threads = True

## Graphing must be done in a thread, if done in the main loop
## it will block event observation of the UI buttons
import threading
thread = threading.Thread(target=graph_currents, args=(lambda: stop_threads, ))
thread.start()

Figure(axes=[Axis(label='Time (sec)', scale=LinearScale()), Axis(label='Current (mA)', orientation='vertical',…

In [71]:
# stop()